# INSTALL LIBRARY

In [28]:
!pip install pandas
! pip3 install pymysql
! pip3 install ipython-sql
! pip3 install mysqlclient
! pip3 install mysql-connector-python

     ---------------------------------------- 9.6/9.6 MB 242.8 kB/s eta 0:00:00
     ------------------------------------ 904.0/904.0 kB 374.0 kB/s eta 0:00:00


In [2]:
import numpy as np 
import sklearn 
import os
import pymysql
import pandas as pd
import mysql.connector

# inisiasi attributes

In [3]:


db_name = "house"
db_host = "localhost"
db_username = "root"
db_password = ""

# Connect to mysql

In [4]:
import mysql.connector
from mysql.connector import Error

try:
    connection = mysql.connector.connect(
        host='localhost',
        user='root',
        password=''
    )

    cursor = connection.cursor()
    cursor.execute("CREATE DATABASE housing_data")
    print("Database created successfully")

except mysql.connector.Error as error:
    print("Error creating database: {}".format(error))
finally:
    if 'connection' in locals() and connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Error creating database: 1007 (HY000): Can't create database 'housing_data'; database exists
MySQL connection is closed


PANGGIL DATA

In [5]:
data_frame = pd.read_csv('~/Downloads/housing.csv')

In [6]:
# masuk kedalam extension mysql

%load_ext sql

In [7]:
# membuat koneksi ke database MySQL

%sql mysql+mysqldb://root@localhost

In [8]:
%sql show databases;

 * mysql+mysqldb://root@localhost
8 rows affected.


Database
data_mahasiswa
housing_data
information_schema
mysql
performance_schema
phpmyadmin
produk1
test


In [9]:
%sql USE housing_data;

 * mysql+mysqldb://root@localhost
0 rows affected.


[]

In [10]:
%%sql
CREATE TABLE housing_data1 (
    longitude FLOAT,
    latitude FLOAT,
    housing_median_age INT,
    total_rooms INT,
    total_bedrooms INT,
    population INT,
    households INT,
    median_income FLOAT,
    median_house_value FLOAT,
    ocean_proximity CHAR
);

 * mysql+mysqldb://root@localhost
(MySQLdb.OperationalError) (1050, "Table 'housing_data1' already exists")
[SQL: CREATE TABLE housing_data1 (
    longitude FLOAT,
    latitude FLOAT,
    housing_median_age INT,
    total_rooms INT,
    total_bedrooms INT,
    population INT,
    households INT,
    median_income FLOAT,
    median_house_value FLOAT,
    ocean_proximity CHAR
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [11]:
%sql drop table housing_data;

 * mysql+mysqldb://root@localhost
(MySQLdb.OperationalError) (1051, "Unknown table 'housing_data.housing_data'")
[SQL: drop table housing_data;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [12]:
%sql show tables;

 * mysql+mysqldb://root@localhost
3 rows affected.


Tables_in_housing_data
housing_data1
housing_details
housing_location


In [13]:

%%sql

LOAD DATA LOCAL INFILE 'housing.csv' 
INTO TABLE housing_data1
FIELDS TERMINATED BY ',' 
ENCLOSED BY '"' 
LINES TERMINATED BY '\n'
IGNORE 1 ROWS;

 * mysql+mysqldb://root@localhost
20640 rows affected.


[]

In [14]:
%%sql 

show variables like 'local_infile'

 * mysql+mysqldb://root@localhost
1 rows affected.


Variable_name,Value
local_infile,ON


# Normalisasi 2NF
Pada contoh di atas, normalisasi 2NF dilakukan dengan membagi data ke dalam dua tabel: housing_location dan housing_details. 

In [15]:

%%sql
-- Tabel pertama: housing_location
CREATE TABLE housing_location (
    location_id INTEGER PRIMARY KEY,
    longitude FLOAT,
    latitude FLOAT,
    ocean_proximity VARCHAR(50)
);

-- Tabel kedua: housing_details
CREATE TABLE housing_details (
    housing_id INTEGER PRIMARY KEY,
    location_id INTEGER,
    housing_median_age INT,
    total_rooms INT,
    total_bedrooms INT,
    population INT,
    households INT,
    median_income FLOAT,
    median_house_value FLOAT,
    FOREIGN KEY (location_id) REFERENCES housing_location(location_id)
);

 * mysql+mysqldb://root@localhost
(MySQLdb.OperationalError) (1050, "Table 'housing_location' already exists")
[SQL: -- Tabel pertama: housing_location
CREATE TABLE housing_location (
    location_id INTEGER PRIMARY KEY,
    longitude FLOAT,
    latitude FLOAT,
    ocean_proximity VARCHAR(50)
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


# Menampilkan Record Pada Tabel

In [16]:
%%sql
SELECT * FROM housing_details;

 * mysql+mysqldb://root@localhost
1 rows affected.


housing_id,location_id,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
1,1,20,100,50,200,80,5000.0,200000.0


In [17]:
%%sql
SELECT * FROM housing_location;

 * mysql+mysqldb://root@localhost
2 rows affected.


location_id,longitude,latitude,ocean_proximity
1,123.45,67.89,Near Beach
2,54.32,12.34,Inland


In [ ]:
%%sql
SELECT * FROM housing_data1;

# Syntax DML dan DDL

Pada contoh implementasi berikut, DDL digunakan untuk membuat tabel housing_location dan housing_details dengan kolom-kolom yang sesuai. Kemudian, DML digunakan untuk memasukkan data ke dalam tabel-tabel tersebut menggunakan perintah INSERT INTO.

%%sql
-- Tabel pertama: housing_location
CREATE TABLE housing_location (
    location_id INTEGER PRIMARY KEY,
    longitude FLOAT,
    latitude FLOAT,
    ocean_proximity VARCHAR(50)
);

-- Tabel kedua: housing_details
CREATE TABLE housing_details (
    housing_id INTEGER PRIMARY KEY,
    location_id INTEGER,
    housing_median_age INT,
    total_rooms INT,
    total_bedrooms INT,
    population INT,
    households INT,
    median_income FLOAT,
    median_house_value FLOAT,
    FOREIGN KEY (location_id) REFERENCES housing_location(location_id)
);

# B. Data Manipulation Language (DML)

DML : data tambahan ditambahkan ke tabel-tabel. 
Tabel housing_location menerima data untuk location_id, longitude, latitude, dan ocean_proximity.

In [18]:


%%sql
-- Insert data into housing_location table
INSERT INTO housing_location (location_id, longitude, latitude, ocean_proximity)
VALUES
    (1, 123.45, 67.89, 'Near Beach'),
    (2, 12.34, 56.78, 'Inland'),
    (3, 98.76, 54.32, 'Near Ocean'),
    (4, 76.54, 32.10, 'Inland'),
    (5, 45.67, 23.45, 'Near Ocean'),
    (6, 87.65, 43.21, 'Inland'),
    (7, 34.56, 65.43, 'Near Beach'),
    (8, 56.78, 12.34, 'Inland'),
    (9, 43.21, 98.76, 'Near Ocean'),
    (10, 23.45, 45.67, 'Inland'),
    (11, 43.21, 87.65, 'Near Beach'),
    (12, 32.10, 76.54, 'Inland'),
    (13, 65.43, 34.56, 'Near Ocean'),
    (14, 12.34, 56.78, 'Near Beach'),
    (15, 54.32, 98.76, 'Inland'),
    (16, 76.54, 43.21, 'Near Ocean'),
    (17, 23.45, 34.56, 'Inland'),
    (18, 45.67, 23.45, 'Near Beach'),
    (19, 43.21, 12.34, 'Inland'),
    (20, 87.65, 65.43, 'Near Ocean'),
    (21, 98.76, 32.10, 'Inland'),
    (22, 65.43, 87.65, 'Near Beach'),
    (23, 34.56, 76.54, 'Inland'),
    (24, 23.45, 45.67, 'Near Ocean'),
    (25, 54.32, 43.21, 'Inland'),
    (26, 12.34, 23.45, 'Near Beach'),
    (27, 32.10, 12.34, 'Inland'),
    (28, 56.78, 54.32, 'Near Ocean'),
    (29, 43.21, 76.54, 'Inland'),
    (30, 65.43, 98.76, 'Near Beach');

-- Insert data into housing_details table
INSERT INTO housing_details (housing_id, location_id, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, median_house_value)
VALUES
    (1, 1, 20, 100, 50, 200, 80, 5000.0, 200000),
    (2, 2, 25, 150, 75, 250, 100, 6000.0, 220000),
    (3, 3, 30, 120, 60, 180, 70, 4500.0, 190000),
    (4, 4, 15, 80, 40, 150, 60, 4000.0, 180000),
    (5, 5, 20, 110, 55, 190, 75, 5500.0, 210000),
    (6, 6, 28, 130, 65, 220, 90, 5200.0, 205000),
    (7, 7, 18, 90, 45, 170, 68, 4800.0, 195000),
    (8, 8, 22, 140, 70, 240, 95, 5700.0, 215000),
    (9, 9, 26, 100, 50, 200, 80, 5100.0, 203000),
    (10, 10, 19, 75, 38, 140, 56, 4200.0, 175000),
    (11, 11, 23, 115, 58, 180, 72, 4700.0, 198000),
    (12, 12, 17, 85, 42, 160, 64, 4300.0, 185000),
    (13, 13, 21, 105, 52, 210, 84, 4900.0, 200000),
    (14, 14, 27, 125, 62, 230, 92, 5400.0, 208000),
    (15, 15, 16, 95, 48, 170, 68, 4600.0, 192000),
    (16, 16, 24, 135, 68, 250, 100, 6100.0, 225000),
    (17, 17, 29, 95, 48, 200, 80, 5100.0, 203000),
    (18, 18, 14, 70, 35, 130, 52, 4000.0, 180000),
    (19, 19, 18, 105, 52, 180, 72, 4700.0, 198000),
    (20, 20, 22, 120, 60, 210, 84, 5200.0, 205000),
    (21, 21, 27, 150, 75, 240, 96, 5800.0, 218000),
    (22, 22, 16, 90, 45, 170, 68, 4500.0, 190000),
    (23, 23, 20, 130, 65, 200, 80, 5000.0, 200000),
    (24, 24, 24, 110, 55, 180, 72, 4800.0, 195000),
    (25, 25, 29, 140, 70, 220, 88, 5500.0, 210000),
    (26, 26, 15, 80, 40, 150, 60, 4300.0, 185000),
    (27, 27, 19, 100, 50, 190, 76, 4700.0, 198000),
    (28, 28, 23, 120, 60, 220, 88, 5200.0, 205000),
    (29, 29, 28, 140, 70, 250, 100, 5900.0, 215000),
    (30, 30, 17, 90, 45, 180, 72, 4500.0, 190000);

 * mysql+mysqldb://root@localhost
(MySQLdb.IntegrityError) (1062, "Duplicate entry '1' for key 'PRIMARY'")
[SQL: -- Insert data into housing_location table
INSERT INTO housing_location (location_id, longitude, latitude, ocean_proximity)
VALUES
    (1, 123.45, 67.89, 'Near Beach'),
    (2, 12.34, 56.78, 'Inland'),
    (3, 98.76, 54.32, 'Near Ocean'),
    (4, 76.54, 32.10, 'Inland'),
    (5, 45.67, 23.45, 'Near Ocean'),
    (6, 87.65, 43.21, 'Inland'),
    (7, 34.56, 65.43, 'Near Beach'),
    (8, 56.78, 12.34, 'Inland'),
    (9, 43.21, 98.76, 'Near Ocean'),
    (10, 23.45, 45.67, 'Inland'),
    (11, 43.21, 87.65, 'Near Beach'),
    (12, 32.10, 76.54, 'Inland'),
    (13, 65.43, 34.56, 'Near Ocean'),
    (14, 12.34, 56.78, 'Near Beach'),
    (15, 54.32, 98.76, 'Inland'),
    (16, 76.54, 43.21, 'Near Ocean'),
    (17, 23.45, 34.56, 'Inland'),
    (18, 45.67, 23.45, 'Near Beach'),
    (19, 43.21, 12.34, 'Inland'),
    (20, 87.65, 65.43, 'Near Ocean'),
    (21, 98.76, 32.10, 'Inland'),
    (2

# Relational Schema

Relational scheme, atau skema relasional, merujuk pada struktur dan organisasi basis data relasional. Skema relasional menggambarkan bagaimana data diorganisir ke dalam tabel,
hubungan antar tabel, dan batasan yang berlaku pada data.

In [19]:
%%sql

SHOW CREATE TABLE housing_location;

 * mysql+mysqldb://root@localhost
1 rows affected.


Table,Create Table
housing_location,"CREATE TABLE `housing_location` ( `location_id` int(11) NOT NULL, `longitude` float DEFAULT NULL, `latitude` float DEFAULT NULL, `ocean_proximity` varchar(50) DEFAULT NULL, PRIMARY KEY (`location_id`)) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci"


In [20]:
%%sql

SHOW CREATE TABLE housing_location;

 * mysql+mysqldb://root@localhost
1 rows affected.


Table,Create Table
housing_location,"CREATE TABLE `housing_location` ( `location_id` int(11) NOT NULL, `longitude` float DEFAULT NULL, `latitude` float DEFAULT NULL, `ocean_proximity` varchar(50) DEFAULT NULL, PRIMARY KEY (`location_id`)) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci"


# Advance Query pada Dataset

Lakukan advance query pada dataset tersebut untuk menemukan insight yang menarik

Menghitung rata-rata median_house_value berdasarkan ocean_proximity:

In [21]:
%%sql
SELECT ocean_proximity, AVG(median_income) AS average_median_income
FROM housing_location
JOIN housing_details ON housing_location.location_id = housing_details.location_id
GROUP BY ocean_proximity;

 * mysql+mysqldb://root@localhost
1 rows affected.


ocean_proximity,average_median_income
Near Beach,5000.0


 Melakukan query untuk menemukan jumlah total ruangan (total_rooms) dan total

In [22]:

%%sql
SELECT housing_median_age, SUM(total_rooms) AS total_rooms, SUM(population) AS total_population
FROM housing_details
GROUP BY housing_median_age
ORDER BY housing_median_age;

 * mysql+mysqldb://root@localhost
1 rows affected.


housing_median_age,total_rooms,total_population
20,100,200


Melakukan query untuk menemukan rata-rata median_income dan median_house_value per jenis ocean_proximity


In [23]:
%%sql
SELECT ocean_proximity, AVG(median_income) AS average_median_income, AVG(median_house_value) AS average_median_house_value
FROM housing_location
JOIN housing_details ON housing_location.location_id = housing_details.location_id
GROUP BY ocean_proximity;

 * mysql+mysqldb://root@localhost
1 rows affected.


ocean_proximity,average_median_income,average_median_house_value
Near Beach,5000.0,200000.0


# User Management

Membuat pengguna user_root dengan hak akses penuh (GRANT ALL PRIVILEGES)

In [24]:
%%sql

GRANT ALL PRIVILEGES ON databasetubes.* TO 'user_root'@'localhost';

 * mysql+mysqldb://root@localhost
0 rows affected.


[]

User Pertama

Tidak dapat melakukan UPDATE

In [25]:
%%sql

GRANT SELECT ON databasetubes.* TO 'user_1'@'localhost';

 * mysql+mysqldb://root@localhost
0 rows affected.


[]

User Kedua

Tidak dapat melakukan CREATE

In [26]:
%%sql

GRANT INSERT, UPDATE, DELETE ON databasetubes.* TO 'user_2'@'localhost';

 * mysql+mysqldb://root@localhost
0 rows affected.


[]

HAK AKSES

User Root

In [27]:
%%sql

SHOW GRANTS FOR 'user_root'@'localhost';

 * mysql+mysqldb://root@localhost
2 rows affected.


Grants for user_root@localhost
GRANT USAGE ON *.* TO `user_root`@`localhost` IDENTIFIED BY PASSWORD '*3DA1F2FC13F5799A662F1B86901FB60D96872C84'
GRANT ALL PRIVILEGES ON `databasetubes`.* TO `user_root`@`localhost`


User Pertama

In [28]:
%%sql

SHOW GRANTS FOR 'user_1'@'localhost';

 * mysql+mysqldb://root@localhost
2 rows affected.


Grants for user_1@localhost
GRANT USAGE ON *.* TO `user_1`@`localhost`
GRANT SELECT ON `databasetubes`.* TO `user_1`@`localhost`


User Kedua

In [29]:
%%sql

SHOW GRANTS FOR 'user_2'@'localhost';

 * mysql+mysqldb://root@localhost
2 rows affected.


Grants for user_2@localhost
GRANT USAGE ON *.* TO `user_2`@`localhost`
"GRANT INSERT, UPDATE, DELETE ON `databasetubes`.* TO `user_2`@`localhost`"
